In [10]:
import torch
import numpy as np
import math
import os
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from torch import optim

In [11]:
FILE_PATH = "camera_localization_task5.csv"
col_names = ["timestamp","qr_num","Cx","Cy","w","h","dist","att"]
df = pd.read_csv(FILE_PATH, header=None, names=col_names)


In [12]:
data = pd.DataFrame.to_numpy(df)
s21x = 37
s20x = 25
s25x = 50.5
s26x = 62.5
s27x = 74.5
s31x = 86.5
s32x = 98
sy = 121.5
f = 547
h0 = 11.5

px = torch.tensor([60.], requires_grad=True)
py = torch.tensor([60.], requires_grad=True)
phi = torch.tensor([0.], requires_grad=True)

timestamp = data[0][0]
currentstamp = 0
print(data.shape)
samples = []

(701, 8)


In [13]:
df

,timestamp,qr_num,Cx,Cy,w,h,dist,att
0,1.603874e+09,31,-151,101,79,76,81.578947,-15.646531
1,1.603874e+09,32,-228,101,79,75,82.666667,-22.923659
2,1.603874e+09,21,164,100,81,76,81.578947,16.919423
3,1.603874e+09,27,-72,101,76,77,80.519481,-7.606749
4,1.603874e+09,25,82,100,77,76,81.578947,8.648222
...,...,...,...,...,...,...,...,...
696,1.603874e+09,32,-228,101,80,75,82.666667,-22.923659
697,1.603874e+09,25,82,101,76,76,81.578947,8.648222
698,1.603874e+09,27,-72,102,76,76,81.578947,-7.606749
699,1.603874e+09,26,5,101,74,76,81.578947,0.531357


Testing 

In [6]:
di = 83.7725
sx = 62.5
sy = 121.5
angle = 0.5

In [7]:
for i in range(100):
    se = di - torch.sqrt((sx - px)*(sx - px) + (sy - py)*(sy - py))
    see = se*se
    see.backward()
    with torch.no_grad():
        px -= px.grad * 0.1
        py -= py.grad * 0.1
        px.grad.zero_()
        py.grad.zero_()

In [8]:
print(px)
print(py)

tensor([59.0465], requires_grad=True)
tensor([37.7987], requires_grad=True)


In [9]:
for i in range(100):
    pse = angle - torch.atan((sy - py.detach())/(sx - px.detach())*(180/math.pi) + phi
    loss = pse*pse
    loss.backward()
    with torch.no_grad():
        phi -= phi.grad * 0.1
        phi.grad.zero_()

SyntaxError: invalid syntax (<ipython-input-9-34be6d40f29c>, line 3)

Using equation 8

In [ ]:
#single first timestamp
angle = []
di = []
samples = []
for i in range(701):
    currentstamp = data[i][0]
    if(currentstamp == timestamp):
        samples.append(data[i])
        angle.append(samples[i][7])
        h = samples[i][5]
        di.append((f*h0)/h)
    else:
        break;

In [ ]:
print(samples[5][0])

In [14]:
px = torch.tensor(60., requires_grad=True, dtype=torch.float)
py = torch.tensor(60., requires_grad=True, dtype=torch.float)
phi = torch.tensor(88., requires_grad=True, dtype=torch.float)
optimizer = optim.SGD([px, py], lr=0.1)

def lossdi(sx, sy, px ,py, di):
    se = di - torch.sqrt((sx - px)*(sx - px) + (sy - py)*(sy - py))
    return se*se

def lossm(sx, sy, px ,py, angle, phi):
    if(angle < 0):
        se = angle - torch.atan((sy - py)/(sx - px))*(180/math.pi) + phi
    else:
        se = angle - (torch.atan((sy - py)/(sx - px))*(180/math.pi) +180 - phi)
    #print(se)
    #print(px , py, phi)
    return se*se

In [15]:

for i in range(1000):
    Totalloss = 0.0
    for i in range(6):
        if(samples[i][1] == 21):
            Totalloss += lossdi(s21x, sy, px, py ,di[i])
            Totalloss += lossm(s21x, sy, px, py ,angle[i], phi)
        elif(samples[i][1] == 20):
            Totalloss += lossdi(s20x, sy, px, py ,di[i])
            Totalloss += lossm(s20x, sy, px, py ,angle[i], phi)
        elif(samples[i][1] == 25):
            Totalloss += lossdi(s25x, sy, px, py ,di[i])
            Totalloss += lossm(s25x, sy, px, py ,angle[i], phi)
        elif(samples[i][1] == 26):
            Totalloss += lossdi(s26x, sy, px, py ,di[i])
            Totalloss += lossm(s26x, sy, px, py ,angle[i], phi)
        elif(samples[i][1] == 27):
            Totalloss += lossdi(s27x, sy, px, py ,di[i])
            Totalloss += lossm(s27x, sy, px, py ,angle[i], phi)      
        elif(samples[i][1] == 31):
            Totalloss += lossdi(s31x, sy, px, py ,di[i])
            Totalloss += lossm(s31x, sy, px, py ,angle[i], phi)
        else:
            Totalloss += lossdi(s32x, sy, px, py ,di[i])
            Totalloss += lossm(s32x, sy, px, py ,angle[i], phi)
    
    #print("Total", Totalloss)
    Totalloss.backward()
    optimizer.step()
    optimizer.zero_grad()
    

IndexError: list index out of range

In [ ]:
print(phi)
print(px)
print(py)

In [255]:
px = torch.tensor(60., requires_grad=True, dtype=torch.float)
py = torch.tensor(60., requires_grad=True, dtype=torch.float)
phi = torch.tensor(180., requires_grad=True, dtype=torch.float)
f = 547
h0 = 11.5


def lossCi(sx, sy, px , py, Ci, f, phi):
    se = Ci - f*torch.tan( torch.atan((sy - py.detach())/(sx - px.detach())) - phi*(math.pi/180) )
    #print("sx", sx, Ci)
    #print(se)
    return se*se

def losshi(sx, sy, px ,py, hi, f, h0):
    e = hi - ((h0*f)/(torch.sqrt((sx - px)*(sx - px) + (sy - py)*(sy - py))))
    return e*e
            

In [266]:
#single first timestamp
Ci = []
hi = []
samples = []
timestamp = data[0][0]
for i in range(701):
    currentstamp = data[i][0]
    if(currentstamp == timestamp):
        samples.append(data[i])
        hi.append(samples[i][5])
        Ci.append(samples[i][2])
    else:
        break;

In [267]:
for i in range(100):
    Totalloss = 0.0
    Totalloss2 = 0.0
    for i in range(6):
        if(samples[i][1] == 21):
            Totalloss2 += lossCi(s21x, sy, px, py , Ci[i], f ,phi)
            Totalloss += losshi(s21x, sy, px, py ,hi[i], f ,h0)
        elif(samples[i][1] == 20):
            Totalloss += losshi(s20x, sy, px, py ,hi[i], f ,h0)
            Totalloss2 += lossCi(s20x, sy, px, py , Ci[i], f ,phi)
        elif(samples[i][1] == 25):
            Totalloss += losshi(s25x, sy, px, py ,hi[i], f ,h0)
            Totalloss2 += lossCi(s25x, sy, px, py , Ci[i], f ,phi)
        elif(samples[i][1] == 26):
            Totalloss += losshi(s26x, sy, px, py ,hi[i], f ,h0)
            Totalloss2 += lossCi(s26x, sy, px, py , Ci[i], f ,phi)
        elif(samples[i][1] == 27):
            Totalloss += losshi(s27x, sy, px, py ,hi[i], f ,h0)
            Totalloss2 += lossCi(s27x, sy, px, py , Ci[i], f ,phi)      
        elif(samples[i][1] == 31):
            Totalloss += losshi(s31x, sy, px, py ,hi[i], f ,h0)
            Totalloss2 += lossCi(s31x, sy, px, py , Ci[i], f ,phi)
        else:
            Totalloss += losshi(s32x, sy, px, py ,hi[i], f ,h0)
            Totalloss2 += lossCi(s32x, sy, px, py , Ci[i], f ,phi)
    
    #print("Total", Totalloss)
    Loss = Totalloss + Totalloss2
    Loss.backward()
    with torch.no_grad():
        px -= px.grad * 0.01
        py -= py.grad * 0.01
        phi -= phi.grad * 0.0001
        phi.grad.zero_()
        px.grad.zero_()
        py.grad.zero_()
    

    

In [268]:
print(px)
print(py)
print(phi)

tensor(60.2173, requires_grad=True)
tensor(42.7963, requires_grad=True)
tensor(-271.6422, requires_grad=True)


In [269]:
def normalizeAngle(phi):
    newAngle = phi
    while (newAngle < 0):
        newAngle = newAngle + 180
    while (newAngle > 360):
        newAngle = newAngle - 360
    print(newAngle)


In [270]:
normalizeAngle(phi)

tensor(88.3578, grad_fn=<AddBackward0>)


# Final estimator

In [208]:
#underconstruction ignorefornow
timestamp = data[0][0]
currentstamp = 0
#single first timestamp
Ci = []
hi = []
px = torch.tensor(60., requires_grad=True, dtype=torch.float)
py = torch.tensor(60., requires_grad=True, dtype=torch.float)
phi = torch.tensor(180., requires_grad=True, dtype=torch.float)
samples = []

for j in range(701):
    currentstamp = data[j][0]
    if(currentstamp == timestamp):
        samples.append(data[j])
    else:
        for epoch in range(100):
            Totalloss = 0.0
            Totalloss2 = 0.0
            for i in range(len(samples)):
                hi = samples[i][5]
                Ci = samples[i][2]
                if(samples[i][1] == 21):
                    Totalloss2 += lossCi(s21x, sy, px, py , Ci, f ,phi)
                    Totalloss += losshi(s21x, sy, px, py ,hi, f ,h0)
                elif(samples[i][1] == 20):
                    Totalloss += losshi(s20x, sy, px, py ,hi, f ,h0)
                    Totalloss2 += lossCi(s20x, sy, px, py , Ci, f ,phi)
                elif(samples[i][1] == 25):
                    Totalloss += losshi(s25x, sy, px, py ,hi, f ,h0)
                    Totalloss2 += lossCi(s25x, sy, px, py , Ci, f ,phi)
                elif(samples[i][1] == 26):
                    Totalloss += losshi(s26x, sy, px, py ,hi, f ,h0)
                    Totalloss2 += lossCi(s26x, sy, px, py , Ci, f ,phi)
                elif(samples[i][1] == 27):
                    Totalloss += losshi(s27x, sy, px, py ,hi, f ,h0)
                    Totalloss2 += lossCi(s27x, sy, px, py , Ci, f ,phi)      
                elif(samples[i][1] == 31):
                    Totalloss += losshi(s31x, sy, px, py ,hi, f ,h0)
                    Totalloss2 += lossCi(s31x, sy, px, py , Ci, f ,phi)
                else:
                    Totalloss += losshi(s32x, sy, px, py ,hi, f ,h0)
                    Totalloss2 += lossCi(s32x, sy, px, py , Ci, f ,phi)
            
            Totalloss.backward()
            with torch.no_grad():
                px -= px.grad * 0.01
                py -= py.grad * 0.01
                px.grad.zero_()
                py.grad.zero_()

            Totalloss2.backward()
            with torch.no_grad():
                phi -= phi.grad * 0.0001
                phi.grad.zero_()
        
        print("Timestamp")
        print(px)
        print(py)
        normalizeAngle(phi)
        print(phi)
                    
        samples = []
        samples.append(data[j])
        timestamp = currentstamp

       
            
    

Timestamp
tensor(60.0136, requires_grad=True)
tensor(42.7998, requires_grad=True)
tensor(88.2168, grad_fn=<AddBackward0>)
tensor(-271.7832, requires_grad=True)
Timestamp
tensor(59.8098, requires_grad=True)
tensor(41.7504, requires_grad=True)
tensor(88.0562, grad_fn=<AddBackward0>)
tensor(-271.9438, requires_grad=True)
Timestamp
tensor(59.5307, requires_grad=True)
tensor(41.4212, requires_grad=True)
tensor(87.8673, grad_fn=<AddBackward0>)
tensor(-272.1327, requires_grad=True)
Timestamp
tensor(59.6218, requires_grad=True)
tensor(41.5937, requires_grad=True)
tensor(87.8987, grad_fn=<AddBackward0>)
tensor(-272.1013, requires_grad=True)
Timestamp
tensor(59.7680, requires_grad=True)
tensor(42.4020, requires_grad=True)
tensor(88.0503, grad_fn=<AddBackward0>)
tensor(-271.9497, requires_grad=True)
Timestamp
tensor(57.4852, requires_grad=True)
tensor(40.9643, requires_grad=True)
tensor(86.7568, grad_fn=<AddBackward0>)
tensor(-273.2432, requires_grad=True)
Timestamp
tensor(58.6204, requires_grad=

Timestamp
tensor(60.7327, requires_grad=True)
tensor(41.6929, requires_grad=True)
tensor(88.6343, grad_fn=<AddBackward0>)
tensor(-271.3657, requires_grad=True)
Timestamp
tensor(61.0403, requires_grad=True)
tensor(41.8638, requires_grad=True)
tensor(88.8160, grad_fn=<AddBackward0>)
tensor(-271.1840, requires_grad=True)
Timestamp
tensor(61.1847, requires_grad=True)
tensor(41.8624, requires_grad=True)
tensor(88.9302, grad_fn=<AddBackward0>)
tensor(-271.0698, requires_grad=True)
Timestamp
tensor(60.7782, requires_grad=True)
tensor(41.7307, requires_grad=True)
tensor(88.6663, grad_fn=<AddBackward0>)
tensor(-271.3337, requires_grad=True)
Timestamp
tensor(60.2125, requires_grad=True)
tensor(41.5819, requires_grad=True)
tensor(88.3181, grad_fn=<AddBackward0>)
tensor(-271.6819, requires_grad=True)
Timestamp
tensor(60.3204, requires_grad=True)
tensor(41.7373, requires_grad=True)
tensor(88.3524, grad_fn=<AddBackward0>)
tensor(-271.6476, requires_grad=True)
Timestamp
tensor(59.9944, requires_grad=

-1.995200412208242